# Issue Valume Predictions for Comparable Markets

This notebook tries to build models and forecast issue volumes for all components in TQ,AQ and TS for 6 countries recognized as comparable markets 

In [1]:
import time
begin = time.ctime()
begin

'Tue Aug 10 14:02:43 2021'

In [2]:
import numpy as np
import pandas as pd
pd.set_option('max.rows',100)

    import matplotlib.pyplot as plt
    %matplotlib inline
    import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

#sklearn
from sklearn.metrics import mean_squared_error

#Time Series
from pylab import rcParams
import statsmodels.api as sm

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

from scipy.stats import boxcox

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
#Params used

ip_path ='./Input/'
op_path ='./Output/'

rcParams['figure.figsize'] = 12, 8


#We have data from Sept 2015, Total months = 69
#Train data length
train_len = 64
#Data to be forecasted or test data length
fc_mths = 5

#Starting month
start_mth = 4 #4-starts from JaN 2016, 28 - starts from Jan 2018

exclude_mons = [] #['01-09-2015','01-10-2015']
exclude_mon_nums=[] #[20150901,20151001]

#Create an empty data frame with our date range with values filled as nan
start = pd.datetime(2015,9,1) #Date from which we have data
end = pd.datetime(2021,5,1) #Current Month  
index = pd.date_range(start, end,freq='MS')

# Create an empty data frame with our date range with values filled as nan
empty_ts = pd.DataFrame(index=index)
empty_ts['Issues'] = np.nan
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from pmdarima.arima import auto_arima

In [3]:
#Params used

ip_path ='./Input/'
op_path ='./Output/'

rcParams['figure.figsize'] = 12, 8


#We have data from Sept 2015, Total months = 69
#Train data length
train_len = 64
#Data to be forecasted or test data length
fc_mths = 5

#Starting month
start_mth = 4 #4-starts from JaN 2016, 28 - starts from Jan 2018

exclude_mons = [] #['01-09-2015','01-10-2015']
exclude_mon_nums=[] #[20150901,20151001]

#Create an empty data frame with our date range with values filled as nan
start = pd.datetime(2015,9,1) #Date from which we have data
end = pd.datetime(2021,5,1) #Current Month  
index = pd.date_range(start, end,freq='MS')

# Create an empty data frame with our date range with values filled as nan
empty_ts = pd.DataFrame(index=index)
empty_ts['Issues'] = np.nan

In [4]:
# Function to Print null counts with counts greater than threshold.
#If threshold is not specified, pritns all null counts
def get_null_cnts(df_, thresh = None):
    null_cnts = df_.isnull().sum()
    null_cnts = null_cnts.sort_values(ascending=False)
    if thresh == None :
        return null_cnts
    else :
        return null_cnts[null_cnts > thresh]

In [5]:
# Function to Print null percentages with percentage greater than threshold.
#If threshold is not specified, pritns all null percentages
def get_null_perc(df_, thresh = None):
    null_perc = round(100 * df_.isnull().sum()/len(df_), 2)
    null_perc = null_perc.sort_values(ascending=False)
    if thresh == None :
        return null_perc
    else :
        return null_perc[null_perc > thresh]

In [6]:
#Function to get the group as test or control
def get_group(x):
    if x in test_cids:
        return 'Test'
    else:
        return 'Control'

In [7]:
#Date handling functions

def convert_dt_str_to_mth(dt_str_):
    dt_ = datetime.strptime(dt_str_,'%d-%m-%Y %H:%M')
    return datetime.strftime(dt_,'%B%Y')

#Format 2020-09-01
def convert_dt_str_to_mth2(dt_str_):
    dt_ = datetime.strptime(dt_str_,'%Y-%m-%d')
    return datetime.strftime(dt_,'%B%Y')


#Date Format:20180701
def convert_dt_int_to_mth(dt_int_):
    dt_ = datetime.strptime(str(dt_int_),'%Y%m%d')
    return datetime.strftime(dt_,'%B%Y')

#Date Format:20180701
def convert_dt_int_to_dt(dt_int_):
    return datetime.strptime(str(dt_int_),'%Y%m%d')

def convert_mth_to_dt(mth_):
    return datetime.strptime(mth_,'%B%Y')

def add_months(dt_, num_):
    return dt_ + relativedelta(months=num_)

def convert_dt_to_mth(dt_):
    return datetime.strftime(dt_,'%B%Y')

def convert_mth_to_date_key(mth_):
    return datetime.strftime(convert_mth_to_dt(mth_),'%Y%m%d')

def get_mth_range(dt_):
    start = add_months(dt_,- (pre_post_months))
    end = add_months(dt_,(pre_post_months))
    dt_range = pd.date_range(start,end,freq='MS')
    mth_range = dt_range.map(convert_dt_to_mth)
    mth_range
    return mth_range

#This function is to get all previous months engaged prior to passed month. To derive if the customer is new in curent month
def get_prev_mths(mth_):
    return eng_dt_range[:eng_dt_range.index(mth_)]

In [8]:
me_df_orig = pd.read_excel('./Input/ME_Analysis.xlsx',sheet_name='Sheet1')

In [9]:
me_df_orig.shape

(29882, 12)

In [10]:
me_df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29882 entries, 0 to 29881
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CalendarMonthStart    29882 non-null  int64 
 1   Editorial Language    29882 non-null  object
 2   Issues                29882 non-null  int64 
 3   EditorialCountryName  29882 non-null  object
 4   ComponentName         29882 non-null  object
 5   QueueCategory         29882 non-null  object
 6   QueueCategory1        29882 non-null  object
 7   QueueCategory2        12680 non-null  object
 8   Country&Lang          29882 non-null  object
 9   Country&Lang(T/F)     147 non-null    object
 10  Only Country          1103 non-null   object
 11  Only Lang             16017 non-null  object
dtypes: int64(2), object(10)
memory usage: 2.7+ MB


In [11]:
me_df_orig.head(2)

,CalendarMonthStart,Editorial Language,Issues,EditorialCountryName,ComponentName,QueueCategory,QueueCategory1,QueueCategory2,Country&Lang,Country&Lang(T/F),Only Country,Only Lang
0,20160101,(blank),1,Australia,Appeals,NQP,AQ,NaN,Australia(blank),NaN,NaN,NaN
1,20160101,(blank),59,China,Appeals,NQP,AQ,NaN,China(blank),NaN,NaN,NaN


In [12]:
#Make a copy before doing any manipulations
me_df = me_df_orig.copy()

In [13]:
#Convert Month Start column to data format
me_df['Month'] = me_df.CalendarMonthStart.map(convert_dt_int_to_dt)

In [14]:
cats_df = me_df[['ComponentName','QueueCategory1','QueueCategory2']].drop_duplicates()

In [15]:
#Components mapped to Tscore
ts_core_comps = list(cats_df[(cats_df.QueueCategory1=='TS') & (cats_df.QueueCategory2=='Tscore')].ComponentName)
ts_core_comps

['Advertiser API',
 'Analytics',
 'BI/Reporting',
 'Campaign Mgmt',
 'Editorial',
 'Microsoft Advertising Editor',
 'Optimization',
 'Product Ads/Shopping',
 'Syndication/Publisher API',
 'Tier 3',
 'UCMA',
 'UCMT',
 'UCMB',
 'UCMC',
 'UCMBA']

In [16]:
#Number of rows mapped to NaN group for TS
me_df[ ((me_df.QueueCategory1=='TS') & (me_df.QueueCategory2.isnull()))].shape

(430, 13)

#### Very few records for nulls in QueueCategory2. We can ignore them safely

In [17]:
#Drop the TS components data not mapped to any group 
me_df = me_df[ ~((me_df.QueueCategory1=='TS') & (me_df.QueueCategory2.isnull()))]

In [18]:
#Introduce a new column with Component Names grouped for TS and as is for AQ and TQ
me_df['Component'] = me_df.ComponentName.replace(ts_core_comps,'Tscore')

In [19]:
#Count of rows by component
me_df.Component.value_counts()

Tscore                        8664
Phishing/Techscam             5808
Appeals                       4019
Complaints                    3480
Accounts & Billing            1841
Delivery                      1679
Supply TQ Investigation        948
Malware/Bundling/Downloads     820
Policy Questions               801
Enablements                    404
Matching                       314
Bing Places                    291
Bing Web Master Tools          205
SEAM                           178
Name: Component, dtype: int64

In [20]:
me_df.head(2)

,CalendarMonthStart,Editorial Language,Issues,EditorialCountryName,ComponentName,QueueCategory,QueueCategory1,QueueCategory2,Country&Lang,Country&Lang(T/F),Only Country,Only Lang,Month,Component
0,20160101,(blank),1,Australia,Appeals,NQP,AQ,NaN,Australia(blank),NaN,NaN,NaN,2016-01-01,Appeals
1,20160101,(blank),59,China,Appeals,NQP,AQ,NaN,China(blank),NaN,NaN,NaN,2016-01-01,Appeals


In [21]:
#Rename EditorialCountryName column to Country 
me_df = me_df.rename(columns={'EditorialCountryName':'Country'})

In [22]:
#Select the columns that we are interested in
me_df = me_df[['Month','Country','Component','Issues']]

In [23]:
print('Number of countries:', len(me_df.Country.unique()), 'Number Of Components', len(me_df.Component.unique()))

Number of countries: 195 Number Of Components 14


In [24]:
#Comparable Market countries list are the ones we consider
cons_cons = ['Denmark', 'Finland', 'France', 'Norway', 'Spain', 'Sweden']
comp_me_df = me_df[me_df.Country.isin(cons_cons)]

In [25]:
#Counts by country
comp_me_df.Country.value_counts()

France     1531
Spain      1061
Sweden      668
Denmark     526
Norway      403
Finland     283
Name: Country, dtype: int64

In [26]:
#Number of Country Component Combinations to consider
con_comp = comp_me_df[['Country','Component']].drop_duplicates().sort_values(['Country','Component'])
print('Number of combinations:', len(con_comp) )

#First and last month for which we have data for a combination
con_comp['FirstMonth'] = np.nan
con_comp['LastMonth'] = np.nan
#Number of months we have data for a combination
con_comp['Total'] = 0
#Add a column for number of missing entries in between min and max of available months
con_comp['Missing'] = 0
#Continuously for how many months we have data from the last month backwards
con_comp['LastCont'] = 0
#Average Isssues per month for a combinations
con_comp['AvgIssues'] = 0

Number of combinations: 75


#### Out of 84Country/Component combinations, we have data for 75 combinations 

In [27]:
comp_me_df.head(2)

,Month,Country,Component,Issues
4,2016-01-01,France,Appeals,1
18,2016-01-01,Denmark,Appeals,10


In [28]:
#Check the data availability for the combinations
for row in con_comp.itertuples():
    con, comp = row[1], row[2]
    temp_df = me_df[(me_df.Country == con) & (me_df.Component == comp)].groupby('Month')['Issues'].sum().reset_index()
    
    #First and Last month for which we have data for 
    first_mon = temp_df.Month.min()
    last_mon = temp_df.Month.max()
    
    #Month Range between first and last months 
    mth_range = pd.date_range( first_mon, last_mon, freq='MS')
    
    #Missing months in the range
    missing_mths = mth_range.difference(temp_df.Month)
    
    if len(missing_mths) > 0:
        #If data missing for a month or more, how many months data do we have continuosly from last month
        last_cont_mths = pd.date_range(max(missing_mths), last_mon, freq='MS')
    else:
        last_cont_mths = mth_range
    
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'FirstMonth'] = convert_dt_to_mth(first_mon)
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'LastMonth'] = convert_dt_to_mth(last_mon)
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'Total'] = len(mth_range)
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'Missing'] = len(missing_mths)
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'LastCont'] = len(last_cont_mths)-1 # Minus one
    
    #Average Issues
    con_comp.loc[(con_comp.Country==con) & (con_comp.Component==comp),'AvgIssues'] = temp_df.Issues.mean()

In [29]:
con_comp['MissingPerc'] = round((con_comp.Missing/con_comp.Total)*100)
con_comp['DataAvl'] = con_comp.Total - con_comp.Missing

In [30]:
con_comp.head(2)

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
16847,Denmark,Accounts & Billing,April2016,May2021,62,19,1,2.069767,31.0,43
18,Denmark,Appeals,January2016,June2021,66,0,65,15.318182,0.0,66


## Filter out combinations without significant data

In [31]:
#Ignore combinations not having more than 15 months of data
cons_con_comp = con_comp[con_comp.DataAvl > 15]
cons_con_comp.shape

(48, 10)

#### 27 combinations are gone if we consider combinations with at least 15 months of data availability

In [32]:
#Ignored combinations are
con_comp[~(con_comp.DataAvl > 15)]

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
29532,Denmark,Bing Places,June2021,June2021,1,0,0,1.000000,0.0,1
4544,Denmark,Enablements,April2019,April2021,25,13,2,1.833333,52.0,12
10075,Denmark,Malware/Bundling/Downloads,February2017,May2021,52,48,1,1.000000,92.0,4
5441,Denmark,Matching,October2019,January2021,16,11,1,1.800000,69.0,5
20,Denmark,Policy Questions,January2016,December2020,60,52,1,1.375000,87.0,8
20961,Finland,Bing Web Master Tools,June2018,June2018,1,0,0,1.000000,0.0,1
22712,Finland,Delivery,March2019,June2021,28,25,1,1.333333,89.0,3
9201,Finland,Malware/Bundling/Downloads,May2016,May2018,25,21,2,1.250000,84.0,4
5953,Finland,Matching,January2020,July2020,7,5,1,1.000000,71.0,2
63,Finland,Policy Questions,January2016,April2019,40,35,1,1.600000,88.0,5


In [33]:
#This can be configred based on the requirement
recent_months = ['June2021','May2021','April2021']
display('Ignored Combinations are:', cons_con_comp[~cons_con_comp.LastMonth.isin(recent_months)])
cons_con_comp = cons_con_comp[cons_con_comp.LastMonth.isin(recent_months)]
cons_con_comp.shape

'Ignored Combinations are:'

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
9048,France,Malware/Bundling/Downloads,March2016,January2021,59,24,1,1.657143,41.0,35
70,France,Policy Questions,January2016,February2021,62,18,2,2.613636,29.0,44
9050,Spain,Malware/Bundling/Downloads,March2016,March2021,61,32,1,1.965517,52.0,29
17234,Sweden,Delivery,September2016,March2021,55,17,7,2.052632,31.0,38
11184,Sweden,Supply TQ Investigation,November2017,March2021,41,15,1,1.692308,37.0,26


(43, 10)

#### Another 5 combinations are gone if we do not 

In [34]:
cons_con_comp

,Country,Component,FirstMonth,LastMonth,Total,Missing,LastCont,AvgIssues,MissingPerc,DataAvl
16847,Denmark,Accounts & Billing,April2016,May2021,62,19,1,2.069767,31.0,43
18,Denmark,Appeals,January2016,June2021,66,0,65,15.318182,0.0,66
19,Denmark,Complaints,January2016,June2021,66,1,54,21.661538,2.0,65
17855,Denmark,Delivery,March2017,June2021,52,23,1,1.689655,44.0,29
8932,Denmark,Phishing/Techscam,January2016,June2021,66,5,22,3.918033,8.0,61
11595,Denmark,Supply TQ Investigation,February2018,June2021,41,20,2,1.809524,49.0,21
16671,Denmark,Tscore,January2016,June2021,66,19,8,2.957447,29.0,47
17756,Finland,Accounts & Billing,March2017,June2021,52,34,1,1.333333,65.0,18
61,Finland,Appeals,January2016,June2021,66,1,59,6.723077,2.0,65
62,Finland,Complaints,January2016,June2021,66,3,47,17.650794,5.0,63


#### By looking at the remaining combination's data, it seems to be dafe to impute missing month's data with zero

## Build models

In [35]:
#Number of combinations to build models for
cons_con_comp.shape

(43, 10)

In [36]:
#Function to calculate error and prepare a data frame
def calc_error(method, act, pred):
    global cons_err_df
 
    #Root Mean Square Error
    rmse = np.sqrt(mean_squared_error(act, pred)).round(2)
    mae = np.mean(abs(act-pred))

#     cons_res_df['Actual_'+method] = results_df_['Issues'] #Not required for every method. Initiate it only once
    cons_res_df['Forecast_'+method] = pred
    
 
    #Mean Absolute Percentage error
    mape = np.round(np.mean(np.abs(act-pred)/act)*100,2)
    
    # Over and Under predictions to check the direction of the error
    over_pred, under_pred = [],[]
    for row in zip(act,pred):
        actual = row[0]
        predicted = row[1]
        if predicted > actual:
            over_pred.append([predicted,actual])
        else:
            under_pred.append([predicted,actual])

    
    #This is to avoid shown nan as average error
    if len(over_pred) == 0:
        over_pred_mape =0
    else:
#         over_pred_avg = round(np.mean(over_pred),1)
        preds = [item[0] for item in over_pred]
        acts = [item[1] for item in over_pred]
        over_pred_mape = np.round(np.mean(np.abs(np.array(preds)-np.array(acts))/np.array(acts))*100,2)

    if len(under_pred) == 0:
        under_pred_mape = 0
    else:
#         under_pred_avg = round(np.mean(under_pred),1)
        preds = [item[0] for item in under_pred]
        acts = [item[1] for item in under_pred]
        under_pred_mape = np.round(np.mean(np.abs(np.array(preds)-np.array(acts))/np.array(acts))*100,2)

    over_preds = str(len(over_pred)) + ' - ' + str(over_pred_mape)
    under_preds = str(len(under_pred)) + ' - ' + str(under_pred_mape)

 
    err_df_ = pd.DataFrame({'Method':[method], 'MAPE': [mape], 'RMSE': [rmse], 'MAE':[mae],
                           'Over Predictions':[over_preds], 'Under Predictions':[under_preds]})
    cons_err_df = cons_err_df.append(err_df_)
    
#     print('Method:', method, 'MAPE:', [mape], 'RMSE:', [rmse], 'MAE:',[mae])
    return err_df_

In [37]:
#Plot results of a time series forecasting model
def plot_results(method_name_, ts_data_train, results_df_):
    plt.figure(figsize=(12,4))
    plt.plot(ts_data_train, label='Train')
    plt.plot(results_df_['Issues'], label='Test')
    plt.plot(results_df_['Issues_FC'], label=method_name_+' Forecast')
    plt.legend(loc='best')
    plt.title(method_name_)
    plt.show()

In [38]:
#Functions to run different models

#Simple Exponential Smoothing
def run_simple_exp_smooth(ts_data_train, ts_data_test, smoothing_level_=0.2, optimized_=False):
    model = SimpleExpSmoothing(ts_data_train)
    try:
        model_fit = model.fit(smoothing_level=smoothing_level_, optimized=optimized_)
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model SES:', str(e))
        return pd.DataFrame()

    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))
    
    return results_df


#Holts method with only trend
def run_holt_trend(ts_data_train, ts_data_test, seasonal_periods_=12 , smoothing_level_=0.2, smoothing_slope_=0.01, optimized_=False):
    
    model = ExponentialSmoothing(np.asarray(ts_data_train) ,seasonal_periods = seasonal_periods_ , trend='additive', seasonal=None)

    try:
        model_fit = model.fit(smoothing_level = smoothing_level_, smoothing_slope = smoothing_slope_, optimized = optimized_)
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model Holt with only Trend:', str(e))
        return pd.DataFrame()
    
    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))
    
    return results_df

#Holt winters with trend and seasonality
def run_holt_wint(ts_data_train, ts_data_test, seasonal_periods_=12 , trend_='add', seas_='add', optimized_=True, use_boxcox_=None,init_method_=None):
    
    model = ExponentialSmoothing(np.asarray(ts_data_train) ,seasonal_periods=seasonal_periods_ ,trend=trend_, seasonal=seas_, 
                                 initialization_method=init_method_, use_boxcox=use_boxcox_)
    
    try:
        model_fit = model.fit(optimized=optimized_)
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model Holt Winters:', str(e))
        return pd.DataFrame()
        
    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))
    
    return results_df

#ARIMA
def run_arima(ts_data_train, ts_data_test, is_bc_=False, order_=(1,0,0)):
    
    model = ARIMA(ts_data_train, order=order_)
    try:
        model_fit = model.fit()
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model ARIMA:', str(e))
        return pd.DataFrame()
    
    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))[0]

    if is_bc_:
        results_df['Issues'] = np.exp(results_df['Issues'])
        results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

    return results_df


#SARIMAX
def run_sarimax(ts_data_train, ts_data_test, is_bc_=False, order_=(1,1,1), seasonal_order_=(1,1,1,12), is_exog_=False, exog_=None):

    promo_train_, promo_test_ = None, None 
    if is_exog_:
        promo_train_ = exog_[0:len(ts_data_train)+1]
        promo_test_ = exog_[len(ts_data_train):]
        print(ts_data_train.shape, ts_data_test.shape, promo_train_.shape, promo_test_.shape)
    
    model = SARIMAX(ts_data_train, order=order_, seasonal_order=seasonal_order_,exog=promo_train_)

    try:
        model_fit = model.fit()
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model SARIMAX:', str(e))
        return pd.DataFrame()
    
    results_df = ts_data_test.to_frame()
    results_df['Issues_FC'] = model_fit.forecast(len(ts_data_test))
    
    if is_bc_:
        results_df['Issues'] = np.exp(results_df['Issues'])
        results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

    return results_df

def run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_d=2, max_p=3, max_q=3,start_P=0, 
                             start_Q=0, max_P=3, max_D=1, max_Q=3, m=12, seasonal=True, trace=True, d=1, D=1, max_order = 5,
                             maxiter = 50,
                             error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100, n_jobs = -1):

    # model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
    model = auto_arima(ts_data_train,start_p=start_p,start_q=start_q,max_p=max_p, max_d=max_d, max_q=max_q,start_P=start_P, 
                             start_Q=start_Q, max_P=max_P, max_D=max_D, max_Q=max_Q, m=m, seasonal=seasonal, trace=trace, d=d, 
                             D=D, max_order = max_order, maxiter = maxiter, 
                             error_action=error_action, suppress_warnings=suppress_warnings, 
                             stepwise=stepwise, random_state=random_state, n_fits=n_fits, n_jobs = n_jobs)


    try:
        model.fit(ts_data_train)
        if debug:
            print(model_fit.params)
    except ValueError as e:
        print('Value Error while fitting the model Auto ARIMA:', str(e))
        return pd.DataFrame()

    results_df = ts_data_test.to_frame()
    
    try:
        results_df['Issues_FC'] = model.predict(len(ts_data_test))
    except ValueError as e:
        print('Value Error while prediction in Auto ARIMA:', str(e))
        return pd.DataFrame()
        
    if is_bc_:
        results_df['Issues'] = np.exp(results_df['Issues'])
        results_df['Issues_FC'] = np.exp(results_df['Issues_FC'])

    return results_df

In [39]:
def plot_calc_error(method, ts_data_train, results_df):
    if len(results_df) == 0:
        print('Results Dataframe is empty')
        return
    if debug:
        plot_results(method, ts_data_train, results_df)
    calc_error(method,results_df['Issues'],results_df['Issues_FC'])

#bc to indicate whether to boxcox transformation
def run_models(ts_data, bc, select='mape'):
    #Train test split - 80:20
    ts_len = len(ts_data)
    train_len = int(ts_len*.8)
    ts_data_train = ts_data[:train_len]
    ts_data_test = ts_data[train_len:]
    
    if bc:
        ts_data_bc = pd.Series(boxcox(ts_data.interpolate(), lmbda=0),name='Issues')
        #Index with all months from first to last
        idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
        #Reset index for boxcox data set
        ts_data_bc.index = idx

        ts_data_train_bc = ts_data_bc[:train_len]
        ts_data_test_bc = ts_data_bc[train_len:]
    
    cons_res_df['Actual'] = ts_data_test

    method = 'SES'
    results_df = run_simple_exp_smooth(ts_data_train, ts_data_test)
    plot_calc_error(method, ts_data_train, results_df)

    method = 'HoltsTrend'
    results_df = run_holt_trend(ts_data_train, ts_data_test)
    plot_calc_error(method, ts_data_train, results_df)

    method = 'HWAdditive'
    results_df = run_holt_wint(ts_data_train, ts_data_test)
    plot_calc_error(method, ts_data_train, results_df)

    if bc:
        method = 'HWMult'
        results_df = run_holt_wint(ts_data_train, ts_data_test,seas_='mul')
        plot_calc_error(method, ts_data_train, results_df)

    ord = (1,1,1)
    method = 'ARIMA_'+str(ord)
    results_df = run_arima(ts_data_train, ts_data_test, order_=ord)
    plot_calc_error(method, ts_data_train, results_df)

    if bc:
        
        ord = (1,1,1)
        method = 'ARIMA_BC_'+str(ord)
        results_df = run_arima(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord) #TODO - pass Box Cox data
        plot_calc_error(method, ts_data_train, results_df)

        ord_1 = (1,1,1)
        ord_2 = (1,1,1,12)
        method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
        results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2)#TODO
        plot_calc_error(method, ts_data_train, results_df)

        ord_1 = (1,2,1)
        ord_2 = (1,1,1,12)
        method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
        results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2) #TODO
        plot_calc_error(method, ts_data_train, results_df)

        ord_1 = (1,2,1)
        ord_2 = (0,1,1,12)
        method = 'SARIMA_BC_'+str(ord_1)+'_'+str(ord_2)
        results_df = run_sarimax(ts_data_train_bc, ts_data_test_bc, is_bc_=True, order_=ord_1, seasonal_order_=ord_2) #TODO
        plot_calc_error(method, ts_data_train, results_df)

    ord_1 = (1,2,1)
    ord_2 = (1,1,1,12)
    method = 'SARIMA_'+str(ord_1)+'_'+str(ord_2)
    results_df = run_sarimax(ts_data_train, ts_data_test, order_=ord_1, seasonal_order_=ord_2)
    plot_calc_error(method, ts_data_train, results_df)


    ord_1 = (1,2,1)
    ord_2 = (0,1,1,12)
    method = 'SARIMA_'+str(ord_1)+'_'+str(ord_2)
    results_df = run_sarimax(ts_data_train, ts_data_test, order_=ord_1, seasonal_order_=ord_2)
    plot_calc_error(method, ts_data_train, results_df)

    method = 'PMD_Diff1'
    results_df = run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_p=8, max_q=8,start_P=0, 
                             start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=1, D=1, 
                             error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
    plot_calc_error(method, ts_data_train, results_df)

    method = 'PMD_Diff2'
    results_df = run_auto_arima(ts_data_train, ts_data_test, is_bc_=False, start_p=1, start_q=1,max_p=8, max_d=3, max_q=8,start_P=0, 
                             start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=2, 
                             error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
    plot_calc_error(method, ts_data_train, results_df)

    if bc:
        method = 'PMD_Diff1_BC'
        results_df = run_auto_arima(ts_data_train_bc, ts_data_test_bc, is_bc_=True, start_p=1, start_q=1,max_p=8, max_q=8,start_P=0, 
                                 start_Q=0, max_P=8, max_Q=8, m=12, seasonal=True, trace=debug, d=1, D=1, 
                                 error_action='warn', suppress_warnings=True, stepwise=True, random_state=20, n_fits=100)
        plot_calc_error(method, ts_data_train, results_df)


In [40]:
%%time
#Forecasting with missing values as 0
debug = False

all_res_dfs = {}
all_err_dfs = {}
i = 0 
for row in cons_con_comp[['Country','Component']].itertuples():
    con, comp = row[1], row[2]
    print('Building models for Country:{} and Component:{}'.format(con, comp))
    print('='*100)
    ts_data = me_df[(me_df.Country==con)&(me_df.Component==comp)].groupby('Month')['Issues'].sum()
    
    #Index with all months from first to last
    idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
    #Reindex time series data to fill values with 0
    ts_data = ts_data.reindex(idx, fill_value=0)
    
    print('TS Data Set shape: ',ts_data.shape)

    #Initiate Consolidated Result and Error dataframes
    cons_err_df = pd.DataFrame()
    cons_res_df = pd.DataFrame()

    run_models(ts_data, bc=False)
    
    
    all_res_dfs[(con, comp)] = cons_res_df
    all_err_dfs[(con, comp)] = cons_err_df
    
    display(cons_err_df)
    if debug:
        display(cons_res_df)
        
    #For testing.Run only limited combos
#     i += 1
#     if i==1:
#         break
    
    print()

Building models for Country:Denmark and Component:Accounts & Billing
TS Data Set shape:  (62,)


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,inf,1.00,0.874954,6 - inf,7 - 30.33
0,HoltsTrend,inf,7.34,6.967840,0 - 0,13 - inf
0,HWAdditive,inf,1.41,1.100947,9 - inf,4 - 31.93
0,"ARIMA_(1, 1, 1)",inf,1.10,0.910991,6 - inf,7 - 22.45
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",inf,1.16,0.993348,5 - inf,8 - 38.5
0,"SARIMA_(1, 2, 1)_(0, 1, 1, 12)",inf,1.22,1.038261,5 - inf,8 - 48.27
0,PMD_Diff1,inf,2.05,1.648219,12 - inf,1 - 40.43
0,PMD_Diff2,inf,8.93,8.017473,0 - 0,13 - inf



Building models for Country:Denmark and Component:Appeals
TS Data Set shape:  (66,)


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,57.91,14.68,12.260772,9 - 72.77,5 - 31.17
0,HoltsTrend,92.08,31.76,25.553342,0 - 0,14 - 92.08
0,HWAdditive,37.70,13.37,10.357144,5 - 53.29,9 - 29.04
0,"ARIMA_(1, 1, 1)",99.40,19.33,17.902362,11 - 118.8,3 - 28.25
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",155.17,33.10,28.672716,13 - 166.39,1 - 9.28
0,"SARIMA_(1, 2, 1)_(0, 1, 1, 12)",155.43,33.17,28.874605,13 - 166.14,1 - 16.23
0,PMD_Diff1,58.84,17.61,12.185418,9 - 76.7,5 - 26.7
0,PMD_Diff2,96.59,22.95,17.001475,12 - 107.41,2 - 31.7



Building models for Country:Denmark and Component:Complaints
TS Data Set shape:  (66,)


...


Building models for Country:Sweden and Component:Phishing/Techscam
TS Data Set shape:  (66,)


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,85.54,3.21,2.466788,8 - 129.81,6 - 26.51
0,HoltsTrend,93.80,3.27,2.542009,8 - 148.46,6 - 20.91
0,HWAdditive,127.88,4.38,3.720093,11 - 157.39,3 - 19.65
0,"ARIMA_(1, 1, 1)",141.71,4.29,3.689305,12 - 161.54,2 - 22.76
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",152.13,5.15,4.532975,12 - 173.07,2 - 26.52
0,"SARIMA_(1, 2, 1)_(0, 1, 1, 12)",152.07,5.17,4.523583,12 - 173.01,2 - 26.42
0,PMD_Diff1,81.73,3.50,2.863234,10 - 98.59,4 - 39.6
0,PMD_Diff2,318.27,11.56,10.648852,13 - 338.07,1 - 60.97



Building models for Country:Sweden and Component:Tscore
TS Data Set shape:  (65,)


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,134.14,4.09,3.514007,8 - 202.0,5 - 25.56
0,HoltsTrend,111.51,6.80,5.676914,2 - 162.32,11 - 102.27
0,HWAdditive,201.01,5.74,5.057349,11 - 235.14,2 - 13.32
0,"ARIMA_(1, 1, 1)",163.13,4.65,4.009259,10 - 203.83,3 - 27.47
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",223.83,6.54,5.843115,12 - 240.67,1 - 21.79
0,"SARIMA_(1, 2, 1)_(0, 1, 1, 12)",288.43,8.65,7.773611,12 - 312.32,1 - 1.72
0,PMD_Diff1,169.95,5.06,4.392755,11 - 196.77,2 - 22.42
0,PMD_Diff2,294.34,9.58,8.464500,13 - 294.34,0 - 0



Wall time: 50min 19s


In [41]:
len(all_err_dfs), len(all_res_dfs), cons_err_df.shape, cons_res_df.shape

(43, 43, (8, 6), (13, 9))

In [42]:
%%time
#Forecasting with missng values linear interpolation
debug = False

all_res_intp_dfs = {}
all_err_intp_dfs = {}
i = 0
for row in cons_con_comp[['Country','Component']].itertuples():
    con, comp = row[1], row[2]
    print('Building models for Country:{} and Component:{}'.format(con, comp))
    print('='*100)
    ts_data = me_df[(me_df.Country==con)&(me_df.Component==comp)].groupby('Month')['Issues'].sum()
    
    #Index with all months from first to last
    idx = pd.date_range(ts_data.index.min(), ts_data.index.max(),freq='MS')
    #Reindex time series data to fill values with nan and use interpolation
    ts_data = ts_data.reindex(idx, fill_value=np.nan)
    #Interploate to impute
    ts_data = ts_data.interpolate()
    
    print('TS Data Set shape: ',ts_data.shape)

    #Initiate Consolidated Result and Error dataframes
    cons_err_df = pd.DataFrame()
    cons_res_df = pd.DataFrame()

    run_models(ts_data, bc=True)
    
    
    all_res_intp_dfs[(con, comp)] = cons_res_df
    all_err_intp_dfs[(con, comp)] = cons_err_df
    
    display(cons_err_df)
    if debug:
        display(cons_res_df)
        
    #For testing.Run only limited combos
#     i += 1
#     if i==1:
#         break
    
    print()

Building models for Country:Denmark and Component:Accounts & Billing
TS Data Set shape:  (62,)


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,41.23,0.78,0.679042,4 - 83.45,9 - 22.47
0,HoltsTrend,40.99,0.77,0.652819,4 - 88.25,9 - 19.99
0,HWAdditive,52.06,1.04,0.775391,6 - 89.83,7 - 19.69
0,HWMult,54.49,1.12,0.871107,5 - 98.83,8 - 26.78
0,"ARIMA_(1, 1, 1)",40.79,0.74,0.612232,5 - 77.72,8 - 17.72
0,"ARIMA_BC_(1, 1, 1)",39.27,0.80,0.710039,4 - 64.84,9 - 27.9
0,"SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)",40.30,0.84,0.725359,5 - 57.7,8 - 29.43
0,"SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)",64.40,1.56,1.381342,1 - 61.56,12 - 64.63
0,"SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)",65.43,1.58,1.397604,1 - 62.88,12 - 65.65
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",106.66,2.22,2.051224,1 - 145.17,12 - 103.45



Building models for Country:Denmark and Component:Appeals
TS Data Set shape:  (66,)


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,57.91,14.68,12.260772,9 - 72.77,5 - 31.17
0,HoltsTrend,92.08,31.76,25.553342,0 - 0,14 - 92.08
0,HWAdditive,37.70,13.37,10.357144,5 - 53.29,9 - 29.04
0,HWMult,40.89,13.87,10.782286,5 - 58.15,9 - 31.31
0,"ARIMA_(1, 1, 1)",99.40,19.33,17.902362,11 - 118.8,3 - 28.25
0,"ARIMA_BC_(1, 1, 1)",49.21,14.70,11.511609,8 - 61.6,6 - 32.7
0,"SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)",60.93,18.36,13.881940,6 - 100.97,8 - 30.91
0,"SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)",134.92,39.51,26.765728,12 - 155.24,2 - 13.04
0,"SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)",187.06,51.03,38.137613,13 - 199.79,1 - 21.45
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",155.17,33.10,28.672716,13 - 166.39,1 - 9.28



Building models for Country:Denmark and Component:Complaints
TS Data Set shape:  (66,)


...


Building models for Country:Sweden and Component:Phishing/Techscam
TS Data Set shape:  (66,)


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,85.54,3.21,2.466801,8 - 129.81,6 - 26.51
0,HoltsTrend,93.75,3.27,2.541615,8 - 148.37,6 - 20.94
0,HWAdditive,126.51,4.34,3.696992,11 - 155.27,3 - 21.08
0,HWMult,116.20,5.22,4.156333,9 - 162.64,5 - 32.62
0,"ARIMA_(1, 1, 1)",140.19,4.25,3.644491,12 - 159.67,2 - 23.35
0,"ARIMA_BC_(1, 1, 1)",143.95,4.39,3.733644,12 - 164.16,2 - 22.68
0,"SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)",121.43,4.78,4.005367,11 - 149.07,3 - 20.06
0,"SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)",96.58,3.97,3.296117,11 - 112.62,3 - 37.77
0,"SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)",97.23,4.02,3.284887,11 - 113.59,3 - 37.22
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",149.89,5.09,4.458966,12 - 170.25,2 - 27.75



Building models for Country:Sweden and Component:Tscore
TS Data Set shape:  (65,)


,Method,MAPE,RMSE,MAE,Over Predictions,Under Predictions
0,SES,136.87,4.13,3.542922,8 - 207.24,5 - 24.27
0,HoltsTrend,146.37,4.31,3.647267,8 - 225.83,5 - 19.24
0,HWAdditive,202.86,5.88,5.191063,11 - 236.63,2 - 17.12
0,HWMult,135.06,4.53,3.891559,10 - 168.77,3 - 22.7
0,"ARIMA_(1, 1, 1)",161.08,4.60,3.964751,9 - 223.27,4 - 21.16
0,"ARIMA_BC_(1, 1, 1)",253.39,7.50,6.395330,12 - 273.49,1 - 12.17
0,"SARIMA_BC_(1, 1, 1)_(1, 1, 1, 12)",214.50,6.91,6.126581,12 - 228.81,1 - 42.73
0,"SARIMA_BC_(1, 2, 1)_(1, 1, 1, 12)",196.36,5.99,5.341595,11 - 228.02,2 - 22.22
0,"SARIMA_BC_(1, 2, 1)_(0, 1, 1, 12)",206.07,6.25,5.645668,12 - 220.03,1 - 38.6
0,"SARIMA_(1, 2, 1)_(1, 1, 1, 12)",250.58,7.40,6.614029,12 - 269.98,1 - 17.73



Wall time: 50min 3s


In [43]:
end = time.ctime()
begin, end

('Tue Aug 10 14:02:43 2021', 'Tue Aug 10 15:43:37 2021')